# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from config import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("Output_Data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kelo,9.3086,15.8066,25.02,75,58,1.32,TD,1689889126
1,1,rongelap,11.1635,166.8941,28.68,81,98,8.16,MH,1689889126
2,2,petauke,-14.2426,31.3253,19.51,46,3,5.48,ZM,1689889126
3,3,port-aux-francais,-49.3500,70.2167,1.74,51,75,16.94,TF,1689888946
4,4,qaqortoq,60.7167,-46.0333,7.78,98,100,1.21,GL,1689889083


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True, 
                                      tiles = "EsriImagery", 
                                      frame_width = 800, 
                                      frame_height = 600,
                                      size = "Humidity", 
                                      scale = 1, 
                                      color = "City"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [53]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 26) &
                            (city_data_df["Max Temp"] > 21) &
                            (city_data_df["Cloudiness"] < 10) &
                            (city_data_df["Wind Speed"] < 5) &
                            (city_data_df["Humidity"] > 70)]

# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,remire-montjoly,4.9167,-52.2667,25.02,88,0,0.00,GF,1689889129
45,45,waingapu,-9.6567,120.2641,22.04,86,1,3.34,ID,1689889130
209,209,nangen,35.4100,127.3858,24.24,89,7,0.86,KR,1689889166
225,225,qulsary,46.9531,54.0198,24.24,71,4,1.81,KZ,1689889169
246,246,hasaki,35.7333,140.8333,25.73,83,1,4.81,JP,1689889172
282,282,grand-bassam,5.2118,-3.7388,24.98,90,7,2.91,CI,1689889177
371,371,madimba,-10.4000,40.3333,23.11,87,0,4.67,TZ,1689889187
521,521,carutapera,-1.1950,-46.0200,24.14,92,7,2.41,BR,1689889091
533,533,espiye,40.9479,38.7115,21.62,82,1,1.77,TR,1689889216
546,546,tayu,-6.5397,111.0518,22.40,85,5,3.99,ID,1689889060


### Step 3: Create a new DataFrame called `hotel_df`.

In [54]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
34,remire-montjoly,GF,4.9167,-52.2667,88,
45,waingapu,ID,-9.6567,120.2641,86,
209,nangen,KR,35.4100,127.3858,89,
225,qulsary,KZ,46.9531,54.0198,71,
246,hasaki,JP,35.7333,140.8333,83,
282,grand-bassam,CI,5.2118,-3.7388,90,
371,madimba,TZ,-10.4000,40.3333,87,
521,carutapera,BR,-1.1950,-46.0200,92,
533,espiye,TR,40.9479,38.7115,82,
546,tayu,ID,-6.5397,111.0518,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel", 
          "apiKey": geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
waingapu - nearest hotel: HOTEL SANDLEWOOD
nangen - nearest hotel: bechora
qulsary - nearest hotel: Камилла
hasaki - nearest hotel: 大新旅館
grand-bassam - nearest hotel: Hôtel 3G
madimba - nearest hotel: No hotel found
carutapera - nearest hotel: No hotel found
espiye - nearest hotel: Bal Otel
tayu - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
34,remire-montjoly,GF,4.9167,-52.2667,88,Complexe Belova
45,waingapu,ID,-9.6567,120.2641,86,HOTEL SANDLEWOOD
209,nangen,KR,35.4100,127.3858,89,bechora
225,qulsary,KZ,46.9531,54.0198,71,Камилла
246,hasaki,JP,35.7333,140.8333,83,大新旅館
282,grand-bassam,CI,5.2118,-3.7388,90,Hôtel 3G
371,madimba,TZ,-10.4000,40.3333,87,No hotel found
521,carutapera,BR,-1.1950,-46.0200,92,No hotel found
533,espiye,TR,40.9479,38.7115,82,Bal Otel
546,tayu,ID,-6.5397,111.0518,85,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [57]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True, 
                                      tiles = "EsriImagery", 
                                      frame_width = 800, 
                                      frame_height = 600,
                                      size = "Humidity", 
                                      scale = 1, 
                                      color = "City"
                                     )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)